In [13]:
import tetra.data.data_loader as dl
import tetra.data.data_processor as dp
from tetra.utils import viewer
import importlib
import matplotlib.pyplot as plt
import os 
from pathlib import Path
import importlib

In [17]:
importlib.reload(dl)

<module 'tetra.data.data_loader' from '/Users/mateobigearddasen/Documents/agrosup_stage/Tetra_Project/src/tetra/data/data_loader.py'>

In [15]:
dataset_path = "../../dataset/sliced"
class_file = "../classes.txt"

In [19]:
#Load dataframe of all the bounding boxes
df = dl.df_bb(dataset_path,splits=["train"])
df["x_center"] = df["x_center"].astype(float)
df["y_center"] = df["y_center"].astype(float)
df["width"] = df["width"].astype(float)
df["height"] = df["height"].astype(float)
print(df.head())

../../dataset/sliced/train
                                                file class_id  x_center  \
0  05150457_JPG.rf.aa3063e76b20d453661107953294a5...        0  0.398438   
1  MFDC6460_JPG.rf.191f2e38c74de83414a9ee901539c0...        0  0.839063   
2  MFDC7343_JPG.rf.76752bc670c8c619bf7ebb6fd1f049...        0  0.827344   
3  05270771_JPG.rf.f3bd15f6adcb844978ac5e6e8c9c2e...        0  0.257812   
4  pptly_bonneveau_1-113-_JPG.rf.f2241256af620280...        0  0.625000   

   y_center     width    height  split  
0  0.239063  0.134375  0.062500  train  
1  0.645312  0.059375  0.056250  train  
2  0.039062  0.051562  0.059375  train  
3  0.288281  0.084375  0.079687  train  
4  0.814844  0.068750  0.060937  train  


In [20]:
df["area"] = df["width"].astype(float) * df["height"].astype(float)
df["aspect_ratioWH"] = df["width"] / df["height"]
df["aspect_ratioHW"] = df["height"] / df["width"]

In [7]:
n = 200
df_sorted = df.sort_values(by="aspect_ratioHW", ascending=False)
criticals = df_sorted.head(n)

In [45]:
#PRINTING
for _, critical in df_invalid.iterrows():
    image_filename = critical["file"]
    label_filename = image_filename.replace(".png", ".txt")
    critical_img = os.path.join(dataset_path, critical["split"], "images",image_filename)
    critical_label = os.path.join(dataset_path, critical["split"], "labels",label_filename)
    print(critical_img)
    image = viewer.view_image(critical_img, critical_label,class_file)
    plt.imshow(image)
    plt.title(critical["file"])
    plt.axis("off")
    print(f" area : {critical["area"]}")
    print(f"Aspect ratio H / W {critical["aspect_ratioHW"]}")
    print(f"Aspect ratio H / W {critical["aspect_ratioWH"]}")
    plt.show()

../../dataset/sliced/train/images/pptly_bonneveau_1-148-_JPG.rf.3b27f9176e7b557ead752293c811db50_637_1536_0_2176_640.png


[ WARN:0@2847.420] global loadsave.cpp:241 findDecoder imread_('../../dataset/sliced/train/images/pptly_bonneveau_1-148-_JPG.rf.3b27f9176e7b557ead752293c811db50_637_1536_0_2176_640.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [39]:
#First
invalid = (
    
        ( 
            (df["aspect_ratioHW"] > 2.75) |
            ( df["aspect_ratioWH"] > 3 )
        
        ) &
    
    (df["area"] < 0.004) &
    (
        (df["x_center"] - df["width"] / 2 < 0.0275) |
        (df["x_center"] + df["width"] / 2 > 0.975) |
        (df["y_center"] - df["height"] / 2 < 0.0275) |
        (df["y_center"] + df["height"] / 2 > 0.975)
    )
)

In [42]:
print(f"Invalid bboxes: {invalid.sum()}")
df_invalid = df[invalid]
df_invalid = df_invalid.sort_values(by="area", ascending=False)

Invalid bboxes: 363


In [43]:
splits = ["train"]
for split in splits: 
    path = Path(dataset_path) / split / "labels"
    
    dp.filter_labels(
        label_dir=path,
        condition_fn=lambda cls, x, y, w, h: not (
            (
            (h / w > 2.75) or
            (w / h > 3)
            ) and
            (h * w < 0.004) and
            (
                (x - w / 2 < 0.0275) or
                (x + w / 2 > 0.975) or
                (y - h / 2 < 0.0275) or
                (y + h / 2 > 0.975)
            )
        )
    )

Deleted empty label: ../../dataset/sliced/train/labels/05060169_JPG.rf.e81933d8d8498938a09e13ee6c69619a_1760_1024_0_1664_640.txt
Deleted empty label: ../../dataset/sliced/train/labels/MFDC1810_JPG.rf.da588ed343b38855c5c7907e84925aab_1605_3584_512_4224_1152.txt
Deleted empty label: ../../dataset/sliced/train/labels/05120441_JPG.rf.c807bd4aa83a0acf2d026e56875715a0_737_1024_512_1664_1152.txt
Deleted empty label: ../../dataset/sliced/train/labels/MFDC5980_JPG.rf.59c8b4dc2d110d6fe9e6e2841537f71a_2471_3584_0_4224_640.txt
Deleted empty label: ../../dataset/sliced/train/labels/04160124_JPG.rf.dc007064c45c70efeeebdf1faa54b9e2_1753_1952_512_2592_1152.txt
Deleted empty label: ../../dataset/sliced/train/labels/04230275_JPG.rf.aef9023d45628dc76396c6027fd0620b_601_1952_512_2592_1152.txt
Deleted empty label: ../../dataset/sliced/train/labels/pptly_bonneveau_1-163-_JPG.rf.9fb4f4246d7891194fefd9bd078583a4_2301_1536_0_2176_640.txt
Deleted empty label: ../../dataset/sliced/train/labels/MFDC8519_JPG.rf.02

In [44]:
for split in splits:
    img_path = Path(dataset_path) / split / "images"
    label_path = Path(dataset_path) / split / "labels"
    dp.remove_imgs_without_annotations(img_path, label_path)

🗑️ Suppression: MFDC6786_JPG.rf.821c111217687e58e551f42b78e5e3fd_107_3584_0_4224_640.png (No annotations)
🗑️ Suppression: MFDC4997_JPG.rf.64c9fd19b17b9477fb5d2f3f2277b336_993_3584_0_4224_640.png (No annotations)
🗑️ Suppression: MFDC8342_JPG.rf.53d348ab72afea084e67f6f74ffbbe91_1116_6144_512_6784_1152.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-271-_JPG.rf.74ac02781294905598816078058b392f_136_1952_512_2592_1152.png (No annotations)
🗑️ Suppression: MFDC6199_JPG.rf.1de9d197f2ec87097e1aaf0d696ed7d7_2209_3584_0_4224_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-21-_JPG.rf.5d5f8642d9c18dcada0ef937c67fa0f1_1342_0_0_640_640.png (No annotations)
🗑️ Suppression: MFDC6722_JPG.rf.a4e51661cde550bd76dec7ae0f3de162_1694_3584_0_4224_640.png (No annotations)
🗑️ Suppression: 05290824_JPG.rf.00479c55473c088508b1fe567de2b400_1864_1536_0_2176_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-206-_JPG.rf.8c52b3ea3d8808605d9fa099ff40b441_1650_512_0_1152_640.png (No annotations